In [1]:
#  Import libraries
import pandas as pd
import sqlite3

#  Load datasets
providers = pd.read_csv("providers_data.csv")
receivers = pd.read_csv("receivers_data.csv")
food_listings = pd.read_csv("food_listings_data.csv")
claims = pd.read_csv("claims_data.csv")

# Quick check
print(providers.head())
print(receivers.head())
print(food_listings.head())
print(claims.head())


   Provider_ID                         Name           Type  \
0            1             Gonzales-Cochran    Supermarket   
1            2  Nielsen, Johnson and Fuller  Grocery Store   
2            3                 Miller-Black    Supermarket   
3            4   Clark, Prince and Williams  Grocery Store   
4            5               Coleman-Farley  Grocery Store   

                                             Address            City  \
0  74347 Christopher Extensions\nAndreamouth, OK ...     New Jessica   
1           91228 Hanson Stream\nWelchtown, OR 27136     East Sheena   
2  561 Martinez Point Suite 507\nGuzmanchester, W...  Lake Jesusview   
3     467 Bell Trail Suite 409\nPort Jesus, IA 61188     Mendezmouth   
4  078 Matthew Creek Apt. 319\nSaraborough, MA 53978   Valentineside   

                Contact  
0       +1-600-220-0480  
1  +1-925-283-8901x6297  
2      001-517-295-2206  
3      556.944.8935x401  
4          193.714.6577  
   Receiver_ID          Name        Ty

In [2]:
# Connect to SQLite DB
conn = sqlite3.connect("food_waste.db")

# Save CSV data to SQL tables
providers.to_sql("Providers", conn, if_exists="replace", index=False)
receivers.to_sql("Receivers", conn, if_exists="replace", index=False)
food_listings.to_sql("Food_Listings", conn, if_exists="replace", index=False)
claims.to_sql("Claims", conn, if_exists="replace", index=False)

print("Database created and tables loaded successfully!")


Database created and tables loaded successfully!


In [3]:
# Drop tables if they already exist
conn.execute("DROP TABLE IF EXISTS Providers;")
conn.execute("DROP TABLE IF EXISTS Receivers;")
conn.execute("DROP TABLE IF EXISTS Food_Listings;")
conn.execute("DROP TABLE IF EXISTS Claims;")

# Create Providers table
conn.execute("""
CREATE TABLE Providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Type TEXT NOT NULL,
    Address TEXT,
    City TEXT NOT NULL,
    Contact TEXT
);
""")

# Create Receivers table
conn.execute("""
CREATE TABLE Receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Type TEXT NOT NULL,
    City TEXT NOT NULL,
    Contact TEXT
);
""")

# Create Food_Listings table
conn.execute("""
CREATE TABLE Food_Listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT NOT NULL,
    Quantity INTEGER NOT NULL,
    Expiry_Date TEXT NOT NULL,
    Provider_ID INTEGER NOT NULL,
    Provider_Type TEXT,
    Location TEXT NOT NULL,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES Providers(Provider_ID)
);
""")

# Create Claims table
conn.execute("""
CREATE TABLE Claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER NOT NULL,
    Receiver_ID INTEGER NOT NULL,
    Status TEXT NOT NULL,
    Timestamp TEXT NOT NULL,
    FOREIGN KEY (Food_ID) REFERENCES Food_Listings(Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES Receivers(Receiver_ID)
);
""")

# Insert data from CSVs into these tables
providers.to_sql("Providers", conn, if_exists="append", index=False)
receivers.to_sql("Receivers", conn, if_exists="append", index=False)
food_listings.to_sql("Food_Listings", conn, if_exists="append", index=False)
claims.to_sql("Claims", conn, if_exists="append", index=False)

print("Database schema created and data inserted!")


Database schema created and data inserted!


In [4]:
queries = {
    # 1
    "providers_receivers_by_city": """
        SELECT City, COUNT(DISTINCT Provider_ID) AS Providers,
               (SELECT COUNT(DISTINCT Receiver_ID) FROM Receivers WHERE City = p.City) AS Receivers
        FROM Providers p
        GROUP BY City;
    """,
    # 2
    "top_provider_type": """
        SELECT Type, COUNT(*) AS Count
        FROM Providers
        GROUP BY Type
        ORDER BY Count DESC
        LIMIT 1;
    """,
    # 3
    "provider_contact_by_city": """
        SELECT Name, Contact
        FROM Providers
        WHERE City = 'YourCityName';
    """,
    # 4
    "top_receivers_by_claims": """
        SELECT r.Name, COUNT(c.Claim_ID) AS Total_Claims
        FROM Claims c
        JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
        GROUP BY r.Name
        ORDER BY Total_Claims DESC;
    """,
    # 5
    "total_food_quantity": """
        SELECT SUM(Quantity) AS Total_Quantity
        FROM Food_Listings;
    """,
    # 6
    "city_with_most_food_listings": """
        SELECT Location, COUNT(*) AS Listings
        FROM Food_Listings
        GROUP BY Location
        ORDER BY Listings DESC
        LIMIT 1;
    """,
    # 7
    "most_common_food_types": """
        SELECT Food_Type, COUNT(*) AS Count
        FROM Food_Listings
        GROUP BY Food_Type
        ORDER BY Count DESC;
    """,
    # 8
    "claims_per_food_item": """
        SELECT f.Food_Name, COUNT(c.Claim_ID) AS Claims
        FROM Claims c
        JOIN Food_Listings f ON c.Food_ID = f.Food_ID
        GROUP BY f.Food_Name;
    """,
    # 9
    "provider_with_most_successful_claims": """
        SELECT p.Name, COUNT(c.Claim_ID) AS Successful_Claims
        FROM Claims c
        JOIN Food_Listings f ON c.Food_ID = f.Food_ID
        JOIN Providers p ON f.Provider_ID = p.Provider_ID
        WHERE c.Status = 'Completed'
        GROUP BY p.Name
        ORDER BY Successful_Claims DESC
        LIMIT 1;
    """,
    # 10
    "claims_status_percentages": """
        SELECT Status,
               ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Claims), 2) AS Percentage
        FROM Claims
        GROUP BY Status;
    """,
    # 11
    "avg_quantity_per_receiver": """
        SELECT r.Name, ROUND(AVG(f.Quantity), 2) AS Avg_Quantity
        FROM Claims c
        JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID
        JOIN Food_Listings f ON c.Food_ID = f.Food_ID
        GROUP BY r.Name;
    """,
    # 12
    "most_claimed_meal_type": """
        SELECT Meal_Type, COUNT(*) AS Claims
        FROM Claims c
        JOIN Food_Listings f ON c.Food_ID = f.Food_ID
        GROUP BY Meal_Type
        ORDER BY Claims DESC
        LIMIT 1;
    """,
    # 13
    "total_donated_by_provider": """
        SELECT p.Name, SUM(f.Quantity) AS Total_Donated
        FROM Food_Listings f
        JOIN Providers p ON f.Provider_ID = p.Provider_ID
        GROUP BY p.Name;
    """,
    # 14
    "top_city_by_completed_claims": """
        SELECT f.Location, COUNT(*) AS Completed_Claims
        FROM Claims c
        JOIN Food_Listings f ON c.Food_ID = f.Food_ID
        WHERE c.Status = 'Completed'
        GROUP BY f.Location
        ORDER BY Completed_Claims DESC
        LIMIT 1;
    """,
    # 15
    "expired_food_items": """
        SELECT Food_Name, Expiry_Date
        FROM Food_Listings
        WHERE DATE(Expiry_Date) < DATE('now');
    """
}

# Function to run all queries
for name, q in queries.items():
    print(f"\n--- {name} ---")
    result = pd.read_sql(q, conn)
    print(result)



--- providers_receivers_by_city ---
                 City  Providers  Receivers
0            Adambury          1          0
1           Adamsview          1          0
2          Adamsville          1          0
3        Aguirreville          1          0
4    Alexanderchester          1          0
..                ...        ...        ...
958       Wrightville          1          0
959         Yatesside          1          0
960      Youngchester          1          0
961      Zimmermanton          1          0
962    Zimmermanville          1          0

[963 rows x 3 columns]

--- top_provider_type ---
          Type  Count
0  Supermarket    262

--- provider_contact_by_city ---
Empty DataFrame
Columns: [Name, Contact]
Index: []

--- top_receivers_by_claims ---
                  Name  Total_Claims
0    William Frederick             5
1         Scott Hunter             5
2         Matthew Webb             5
3       Anthony Garcia             5
4      Kristine Martin             4
